In [3]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [4]:
if not os.path.exists('temp'):
    os.mkdir('temp')

In [5]:
from langchain.tools import tool
from typing import List, Dict
from langchain_teddynote.tools import GoogleNews
from langchain_community.agent_toolkits import FileManagementToolkit

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_teddynote.messages import AgentStreamParser

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [6]:
@tool
def lastest_news(k: int = 5) -> List[Dict[str, str]]:
    """Look up latest news"""

    news_tool = GoogleNews()
    
    return news_tool.search_latest(k=k) 

In [7]:
answer = lastest_news.invoke({'k': 2})

for ans in answer:
    print(ans)

{'url': 'https://news.google.com/rss/articles/CBMickFVX3lxTFBtRzB3YmZMX3BFNkRMXzd2RVZHUUNiN3RhU015ZGt1bFJHdmdQRVpDeUdkLXp4cGFHWmlKc2kxVVpLVXNsdV90YnpCdTdUc05aV0JIUDc5ZWtmZjF1MnBNcTcyWW14X0U5WlFoTkpFOFpUdw?oc=5', 'content': '[단독] 경찰, ‘내란’ 혐의 한덕수·최상목 출국금지 했다 - 한겨레'}
{'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTE53dW1WblQ4ZlZVMnZvbzA0X2RRUl9QMG9HVTZzNkVEMnN3dTVBSTZTQTJaSlJKZTNpNUg3WGl2NnRteU5RenFtbXBpMGU4dFNoOGx4d2tvcGhLZ9IBX0FVX3lxTE9ZdnAwbjZTVFBmZlE2TjBHaDl3dVNVdG1wbFpTd19HNjUzZDlNSjJPalNDZDhKNGl4TjV2X3YyWnBTUl9MTk9vbUlzNzRtSTJ3aVdka2F0TWI1bGFrTU1R?oc=5', 'content': '[여론조사 ‘경향’] 이재명 하락폭보다 김문수 상승폭이 더 높았다···지지율 격차 더 좁혀져 - 경향신문'}


In [8]:
working_directory = 'temp'

toolkit = FileManagementToolkit(root_dir=str(working_directory))
tools = toolkit.get_tools()

In [9]:
tools

[CopyFileTool(root_dir='temp'),
 DeleteFileTool(root_dir='temp'),
 FileSearchTool(root_dir='temp'),
 MoveFileTool(root_dir='temp'),
 ReadFileTool(root_dir='temp'),
 WriteFileTool(root_dir='temp'),
 ListDirectoryTool(root_dir='temp')]

In [10]:
tools.append(lastest_news)

In [11]:
tools

[CopyFileTool(root_dir='temp'),
 DeleteFileTool(root_dir='temp'),
 FileSearchTool(root_dir='temp'),
 MoveFileTool(root_dir='temp'),
 ReadFileTool(root_dir='temp'),
 WriteFileTool(root_dir='temp'),
 ListDirectoryTool(root_dir='temp'),
 StructuredTool(name='lastest_news', description='Look up latest news', args_schema=<class 'langchain_core.utils.pydantic.lastest_news'>, func=<function lastest_news at 0x000002633F1EB1A0>)]

In [12]:
for tool in tools:
    print(f'{tool.name} : {tool.description}')

copy_file : Create a copy of a file in a specified location
file_delete : Delete a file
file_search : Recursively search for files in a subdirectory that match the regex pattern
move_file : Move or rename a file from one location to another
read_file : Read file from disk
write_file : Write file to disk
list_directory : List files and directories in a specified folder
lastest_news : Look up latest news


In [13]:
llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini'
)

In [14]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            'system', 
            'You are a helpful assistant. '
            'Make sure to use the `lastest_news` tool to find lastest news. '
            'Make sure to use the `file_managent` tool to manage files. '
        ),
        ('placeholder', '{chat_history}'),
        ('human', '{input}'),
        ('placeholder', '{agent_scratchpad}')
    ]
)

In [15]:
agent = create_tool_calling_agent(llm, tools, prompt)      

In [16]:
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=False, 
    max_iterations=10, 
    max_execution_time=10, 
    handle_parsing_errors=True
)

In [ ]:
store = {}

def get_session_history(session_ids):  # session_ids ->'abc123'
    if session_ids not in store:
        store[session_ids] = ChatMessageHistory()
    
    return store[session_ids]

In [19]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor, # 에이전트 실행기
    get_session_history, # 대화내용 기록 메모리 함수
    input_messages_key='input',  # 사용자 질문
    history_messages_key='chat_history'
)

In [20]:
agent_stream_parser = AgentStreamParser()